<a href="https://colab.research.google.com/github/kumarchandan/EIP4/blob/master/EIP4_S5_AssignmentPersonAttrubutes_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
 # mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


In [0]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

In [0]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

In [0]:
import keras
import numpy as np

img_rows, img_cols = 224, 224

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True, augmentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation = augmentation

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        if self.augmentation is not None:
          image = self.augmentation.flow(images, shuffle=False).next()
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [0]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.2)
train_df.shape, val_df.shape

In [0]:
train_df.head()

In [0]:
train_df.iloc[1370]

In [0]:
from IPython.display import Image
path = train_df.image_path[1370]
Image(filename=path)

In [0]:
# create train and validation data generators
# Use some data augmentation
train_gen = PersonDataGenerator(
    train_df,
    batch_size=32)

valid_gen = PersonDataGenerator(
    val_df,
    batch_size=64,
    shuffle=False
  )

In [0]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

In [0]:
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
import os

In [0]:
# backbone = VGG16(
#     weights="imagenet", 
#     include_top=False, 
#     input_tensor=Input(shape=(224, 224, 3))
# )
backbone = Sequential()

# 1st set of CONV-ReLU Layers
backbone.add(Convolution2D(32, 3, 3, activation='relu',  use_bias=False,input_shape=(224,224,3)))
backbone.add(BatchNormalization())
backbone.add(Dropout(0.1))

# 2nd set of CONV-ReLU Layers
backbone.add(Convolution2D(32, 3, 3, use_bias=False,activation='relu'))
backbone.add(BatchNormalization())
backbone.add(Dropout(0.1))

# 3rd set of CONV-ReLU Layers
backbone.add(Convolution2D(32, 1, 1,use_bias=False, activation='relu'))
backbone.add(MaxPooling2D(pool_size=(2, 2)))

# 4th set of CONV-ReLU Layers
backbone.add(Convolution2D(32, 3, 3, use_bias=False,activation='relu'))
backbone.add(BatchNormalization())
backbone.add(Dropout(0.1))

# 5th set of CONV-ReLU Layers
backbone.add(Convolution2D(16, 3, 3,use_bias=False, activation='relu'))
backbone.add(BatchNormalization())
backbone.add(Dropout(0.1))

# 6th set of CONV-ReLU Layers
backbone.add(Convolution2D(16, 3, 3,use_bias=False, activation='relu'))
backbone.add(BatchNormalization())
backbone.add(Dropout(0.1))

# 7th set of CONV-ReLU Layers
backbone.add(Convolution2D(16, 3, 3,use_bias=False, activation='relu'))
backbone.add(BatchNormalization())
backbone.add(Dropout(0.1))

# 8th set of CONV-ReLU Layers
backbone.add(Convolution2D(10, 4, 4,use_bias=False,activation='relu'))
backbone.add(BatchNormalization())
backbone.add(Dropout(0.1))

backbone.add(MaxPooling2D(pool_size=(2, 2)))

# 9th set of CONV-ReLU Layers
backbone.add(Convolution2D(32, 3, 3, use_bias=False,activation='relu'))
backbone.add(BatchNormalization())
backbone.add(Dropout(0.1))
backbone.add(MaxPooling2D(pool_size=(2, 2)))

# 10th set of CONV-ReLU Layers
backbone.add(Convolution2D(32, 3, 3, use_bias=False,activation='relu'))
backbone.add(BatchNormalization())
backbone.add(Dropout(0.1))

backbone.add(MaxPooling2D(pool_size=(2, 2)))

# 11th set of CONV-ReLU Layers
backbone.add(Convolution2D(32, 3, 3, use_bias=False,activation='relu'))
backbone.add(BatchNormalization())
backbone.add(Dropout(0.1))

backbone.add(Convolution2D(10, 1,1, activation='relu'))



neck = backbone.output
neck = Flatten(name="flatten")(neck)
neck = Dense(512, activation="relu")(neck)


def build_tower(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.3)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

#
model.summary()

In [0]:
# losses = {
# 	"gender_output": "binary_crossentropy",
# 	"image_quality_output": "categorical_crossentropy",
# 	"age_output": "categorical_crossentropy",
# 	"weight_output": "categorical_crossentropy",

# }
# loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0}
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

def scheduler(epoch, lr):
  lr = 1e-3
  if epoch > 150:
    lr = round(0.1 * 1/(1 + 0.319 * epoch), 10)
  elif epoch > 110:
    lr = round(0.1 * 1/(1 + 0.319 * epoch), 10)
  elif epoch > 20:
    lr = 1e-3
  elif epoch > 0:
    lr = 1e-2
  
  print('Learning rate: ', lr)
  return lr

opt=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
    # loss_weights=loss_weights, 
    metrics=["accuracy"]
)

In [0]:
# model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=10)

In [0]:


# Checkpoints
model_type='Assign5'
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
gender_filepath = os.path.join(save_dir, "gender.h5")
imgQuality_filepath = os.path.join(save_dir, "imgQuality.h5")
age_filepath = os.path.join(save_dir, "age.h5")
weight_filepath = os.path.join(save_dir, "weight.h5")
bag_filepath = os.path.join(save_dir, "bag.h5")
footwear_filepath = os.path.join(save_dir, "footwear.h5")
pose_filepath = os.path.join(save_dir, "pose.h5")
emotion_filepath = os.path.join(save_dir, "emotion.h5")

gender_checkpoint = ModelCheckpoint(filepath=gender_filepath,
                             monitor='val_gender_output_acc',
                             verbose=1,
                             save_best_only=True)
imgQuality_checkpoint = ModelCheckpoint(filepath=imgQuality_filepath,
                             monitor='val_image_quality_output_acc',
                             verbose=1,
                             save_best_only=True)
age_checkpoint = ModelCheckpoint(filepath=age_filepath,
                             monitor='val_age_output_acc',
                             verbose=1,
                             save_best_only=True)
weight_checkpoint = ModelCheckpoint(filepath=weight_filepath,
                             monitor='val_weight_output_acc',
                             verbose=1,
                             save_best_only=True)
bag_checkpoint = ModelCheckpoint(filepath=bag_filepath,
                             monitor='val_bag_output_acc',
                             verbose=1,
                             save_best_only=True)
footwear_checkpoint = ModelCheckpoint(filepath=footwear_filepath,
                             monitor='val_footwear_output_acc',
                             verbose=1,
                             save_best_only=True)
pose_checkpoint = ModelCheckpoint(filepath=pose_filepath,
                             monitor='val_pose_output_acc',
                             verbose=1,
                             save_best_only=True)
emotion_checkpoint = ModelCheckpoint(filepath=emotion_filepath,
                             monitor='val_emotion_output_acc',
                             verbose=1,
                             save_best_only=True)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
#                               factor = 0.2,
#                               patience = 3,
#                               verbose = 1,
#                               min_delta = 0.00001)


lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

lr_scheduler = LearningRateScheduler(scheduler, verbose=1) #LearningRateScheduler(lr_schedule)

callbacks = [earlystop, lr_reducer, lr_scheduler]

model.compile(
    optimizer='adam',
    loss="categorical_crossentropy", 
    # loss_weights=loss_weights, 
    metrics=["accuracy"]
)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=100,
    verbose=1
)